# 4.1 Connect LLM to LangChain

LangChain provides an efficient development framework for developing custom applications based on LLM, which allows developers to quickly stimulate the powerful capabilities of LLM and build LLM applications. LangChain also supports a variety of large models, and has built-in calling interfaces for large models such as OpenAI and LLAMA. However, LangChain does not have all large models built in. It provides strong scalability by allowing users to customize LLM types.

## 1. Call ChatGPT based on LangChain

LangChain provides encapsulation for a variety of large models. Based on the LangChain interface, ChatGPT can be easily called and integrated into personal applications built on the LangChain framework. Here we briefly describe how to use the LangChain interface to call ChatGPT.

Note that calling ChatGPT based on the LangChain interface also requires configuring your personal key, and the configuration method is the same as above.

### 1.1 Models
Import `OpenAI`'s dialogue model `ChatOpenAI` from `langchain.chat_models`. In addition to OpenAI, `langchain.chat_models` also integrates other dialogue models. For more details, please refer to the [Langchain official documentation](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.chat_models).

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable OPENAI_API_KEY
openai_api_key = os.environ['OPENAI_API_KEY']

If langchain-openai is not installed, please run the following code first!

In [6]:
from langchain_openai import ChatOpenAI

Next you need to instantiate a ChatOpenAI class. You can pass in hyperparameters to control the answer, such as the `temperature` parameter.

In [8]:
# Here we set the parameter temperature to 0.0 to reduce the randomness of the generated answers.
# If you want to get different and innovative answers every time, you can try adjusting this parameter.
llm = ChatOpenAI(temperature=0.0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001B17F799BD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001B17F79BA60>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatgptid.net/v1', openai_proxy='')

The cell above assumes that your OpenAI API key is set in an environment variable. If you wish to manually specify your API key, use the following code:

In [14]:
llm = ChatOpenAI(temperature=0, openai_api_key="YOUR_API_KEY")

As you can see, the ChatGPT-3.5 model is called by default. In addition, several commonly used hyperparameter settings include:

· model_name: the model to be used, the default is ‘gpt-3.5-turbo’, and the parameter settings are consistent with the OpenAI native interface parameter settings.

· temperature: temperature coefficient, the value is the same as the native interface.

· openai_api_key: OpenAI API key, if you do not use environment variables to set the API Key, you can also set it during instantiation.

· openai_proxy: Set the proxy, if you do not use environment variables to set the proxy, you can also set it during instantiation.

· streaming: whether to use streaming, that is, output the model answer word by word, the default is False, which is not repeated here.

· max_tokens: the maximum number of tokens output by the model, the meaning and value are the same as above.

Once we have initialized the `LLM` of your choice, we can try using it! Let's ask "Please tell me about yourself!"

In [10]:
output = llm.invoke("请你自我介绍一下自己！")

In [11]:
output

AIMessage(content='你好，我是一个智能助手，专注于为用户提供各种服务和帮助。我可以回答问题、提供信息、解决问题，帮助用户更高效地完成工作和生活。如果您有任何疑问或需要帮助，请随时告诉我，我会尽力帮助您。感谢您的使用！', response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 20, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

### 1.2 Prompt (Prompt Template)

When we develop large model applications, most of the time we don't pass user input directly to the LLM. Usually, they add the user input to a larger text, called a `prompt template`, which provides additional context about the specific task at hand.
PromptTemplates help with this! They bundle all the logic from user input to a fully formatted prompt. This can start very simple - for example, the prompt that generates the above string is:

We need to construct a personalized Template first:

In [10]:
from langchain_core.prompts import ChatPromptTemplate

# Here we ask the model to translate the given text into Chinese
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

Next, let's take a look at the complete prompt template that has been constructed:

In [12]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

We know that the interface of the chat model is based on messages, not raw text. PromptTemplates can also be used to generate message lists. In this example, `prompt` contains not only the input content information, but also the information of each `message` (role, position in the list, etc.). Usually, a `ChatPromptTemplate` is a list of `ChatMessageTemplate`. Each `ChatMessageTemplate` contains instructions for formatting the chat message (its role and content).

Let's take a look at an example:

In [7]:
from langchain.prompts.chat import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)
messages

[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.'),
 HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。')]

Next, let's call the defined `llm` and `messages` to output the answer:

In [9]:
output  = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.')

### 1.3 Output parser

OutputParsers convert the raw output of the language model into a format that can be used downstream. There are several main types of OutputParsers, including:
- Convert LLM text to structured information (such as JSON)
- Convert ChatMessage to string
- Convert extra information returned by calls other than messages (such as OpenAI function calls) to string

Finally, we pass the model output to `output_parser`, which is a `BaseOutputParser`, which means it accepts **strings or BaseMessages as input**. StrOutputParser in particular simply converts any input to a string.

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

As can be seen from the above results, we successfully parsed the output of type `ChatMessage` into `string` through the output parser

### 1.4 Complete Process

We can now combine all of this into a chain. This chain will take input variables, pass those variables to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser. Next we'll use the LCEL syntax to quickly implement a chain. Let's see it in action!

In [12]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})


'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

Let’s test another example:

In [14]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language":"英文", "output_language":"中文","text": text})

'我扛着比我的身体还重的行李，潜入尼罗河的底部。穿过几道闪电后，我看到一堆光环，不确定这是否就是目的地。'

> What is LCEL? 
LCEL (LangChain Expression Language) is a new syntax and an important addition to the LangChain toolkit. It has many advantages that make it easier and more convenient to work with LangChain and agents.

- LCEL provides asynchronous, batch, and stream processing support, allowing code to be quickly ported across different servers.
- LCEL has fallback measures to solve the problem of LLM format output.
- LCEL increases the parallelism of LLM and improves efficiency.
- LCEL has built-in logging, which helps understand the operation of complex chains and agents even when agents become complex.

Usage example:

`chain = prompt | model | output_parser`

In the above code, we use LCEL to piece together different components into a chain, in which user input is passed to the prompt template, then the prompt template output is passed to the model, and then the model output is passed to the output parser. The symbol | is similar to the Unix pipe operator, which links different components together and uses the output of one component as the input of the next component.

## 2. Use LangChain to call Baidu Wenxin Yiyan

We can also call Baidu Wenxin model through the LangChain framework to integrate the Wenxin model into our application framework.

### 2.1 Customize LLM to access langchain
In the old version, LangChain does not directly support Wenxin calls, and we need to customize an LLM that supports Wenxin model calls. Here, in order to show users how to customize LLM, we briefly describe this method in "Appendix 1. LangChain Custom LLM", and you can also refer to [source document](https://python.langchain.com/v0.1/docs/modules/model_io/llms/custom_llm/).

Here, we can directly call the customized Wenxin_LLM. For details on how to encapsulate Wenxin_LLM, see `wenxin_llm.py`.

**Note: The following code needs to download our encapsulated code [wenxin_llm.py](./wenxin_llm.py) to the same directory as this Notebook before it can be used directly. Because the new version of LangChain can directly call the Wenxin Qianfan API, we recommend using the code in the next section to call the Wenxin Yiyan model**

In [1]:
# Need to download source code
from wenxin_llm import Wenxin_LLM

We want to store the secret key directly in the .env file and load it into the environment variable like calling ChatGPT, so as to hide the specific details of the secret key and ensure security. Therefore, we need to configure `QIANFAN_AK` and `QIANFAN_SK` in the .env file and load it using the following code:

In [2]:
from dotenv import find_dotenv, load_dotenv
import os

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable API_KEY
wenxin_api_key = os.environ["QIANFAN_AK"]
wenxin_secret_key = os.environ["QIANFAN_SK"]

In [3]:
llm = Wenxin_LLM(api_key=wenxin_api_key, secret_key=wenxin_secret_key, system="你是一个助手！")

In [9]:
llm.invoke("你好，请你自我介绍一下！")

[INFO] [03-31 22:12:53] openapi_requestor.py:316 [t:27812]: requesting llm api endpoint: /chat/eb-instant


1
2


'你好！我是助手，负责协助您完成各种任务。我具备快速响应、高效执行和灵活适应的能力，致力于为您提供优质的服务。无论您需要什么帮助，我都会尽力满足您的需求。'

In [8]:
# Or use
llm(prompt="你好，请你自我介绍一下！")

[INFO] [03-31 22:12:41] openapi_requestor.py:316 [t:27812]: requesting llm api endpoint: /chat/eb-instant


1
2


'你好！我是助手，负责协助您完成各种任务。我具备快速学习和处理信息的能力，能够根据您的需求提供帮助和回答问题。无论您需要什么帮助，我都会尽力提供支持。'

Therefore, we can add the Wenxin model to the LangChain architecture and implement the call of the Wenxin model in the application.

### 2.2 Directly call Wenxinyiyan in langchain

We can also use the new version of LangChain to directly call the Wenxinyiyan model.

In [ ]:

from dotenv import find_dotenv, load_dotenv
import os

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable API_KEY
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]


In [ ]:
#Install required dependencies
%pip install -qU langchain langchain-community

In [4]:
from langchain_community.llms import QianfanLLMEndpoint

llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好，请你自我介绍一下！")
print(res)

d:\Miniconda\miniconda3\envs\llm2\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
[INFO] [03-31 22:40:14] openapi_requestor.py:316 [t:3684]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-31 22:40:14] oauth.py:207 [t:3684]: trying to refresh access_token for ak `MxBM7W***`
[INFO] [03-31 22:40:15] oauth.py:220 [t:3684]: sucessfully refresh access_token


你好！我是文心一言，英文名是ERNIE Bot。我是一款人工智能语言模型，可以协助你完成范围广泛的任务并提供有关各种主题的信息，比如回答问题，提供定义和解释及建议，还能提供上下文知识和对话管理。如果你有任何问题或需要帮助，随时向我提问，我会尽力回答。


## 3. iFlytek Spark

We can also call the iFlytek Spark model through the LangChain framework. For more information, refer to [SparkLLM](https://python.langchain.com/docs/integrations/llms/sparkllm)

We want to store the secret key directly in the .env file and load it into the environment variable like calling ChatGPT, so as to hide the specific details of the secret key and ensure security. Therefore, we need to configure `IFLYTEK_SPARK_APP_ID`, `IFLYTEK_SPARK_API_KEY` and `IFLYTEK_SPARK_API_SECRET` in the .env file and load it using the following code:

In [5]:
from dotenv import find_dotenv, load_dotenv
import os

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable API_KEY
IFLYTEK_SPARK_APP_ID = os.environ["IFLYTEK_SPARK_APP_ID"]
IFLYTEK_SPARK_API_KEY = os.environ["IFLYTEK_SPARK_API_KEY"]
IFLYTEK_SPARK_API_SECRET = os.environ["IFLYTEK_SPARK_API_SECRET"]

In [6]:
def gen_spark_params(model):
    '''
    构造星火模型请求参数
    '''

    spark_url_tpl = "wss://spark-api.xf-yun.com/{}/chat"
    model_params_dict = {
# v1.5 version
        "v1.5": {
            "domain": "general", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v1.1") # 云端环境的服务地址
        },
# v2.0 version
        "v2.0": {
            "domain": "generalv2", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v2.1") # 云端环境的服务地址
        },
# v3.0 version
        "v3.0": {
            "domain": "generalv3", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v3.1") # 云端环境的服务地址
        },
# v3.5 version
        "v3.5": {
            "domain": "generalv3.5", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v3.5") # 云端环境的服务地址
        }
    }
    return model_params_dict[model]

In [7]:
from langchain_community.llms import SparkLLM

spark_api_url = gen_spark_params(model="v1.5")["spark_url"]

# Load the model (v3.0 is used by default)
llm = SparkLLM(spark_api_url = spark_api_url)  #指定 v1.5版本

In [11]:
res = llm("你好，请你自我介绍一下！")
print(res)

您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。


Therefore, we can add the Spark model to the LangChain architecture and implement the call to the Wenxin model in the application.

## 4. Use LangChain to call Zhipu GLM

We can also call the Zhipu AI big model through the LangChain framework to connect it to our application framework. Since the [ChatGLM](https://python.langchain.com/docs/integrations/llms/chatglm) provided in langchain is no longer available, we need to customize an LLM.

If you are using the Zhipu GLM API, you need to download our encapsulated code [zhipuai_llm.py](./zhipuai_llm.py) to the same directory as this Notebook before you can run the following code to use GLM in LangChain.

According to Zhipu’s official announcement, the following models will be deprecated soon. After these models are deprecated, they will be automatically routed to new models. Please note that before the deprecation date, update your model code to the latest version to ensure a smooth transition of services. For more information about the model, please visit [model](https://open.bigmodel.cn/dev/howuse/model)

| Model code | Deprecation date | Point to model |
| ---- | ---- | ---- |
|chatglm_pro|December 31, 2024|glm-4|
|chatglm_std|December 31, 2024|glm-3-turbo|
|chatglm_lite|December 31, 2024|glm-3-turbo|

### 4.1 Customize chatglm to access langchain

In [1]:
from zhipuai_llm import ZhipuAILLM

In [2]:

from dotenv import find_dotenv, load_dotenv
import os

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

In [7]:
zhipuai_model = ZhipuAILLM(model = "glm-4", temperature = 0.1, api_key = api_key)  #model="glm-4-0520", 

In [8]:
zhipuai_model("你好，请你自我介绍一下！")

'你好！我是智谱清言，是清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。'